In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def CRMSLE(truth, pred): 
    rmsle1 = (mean_squared_log_error(truth[:,0], pred[:,0]))**(1/2) 
    rmsle2 = (mean_squared_log_error(truth[:,1], pred[:,1]))**(1/2) 
    rmsle3 = (mean_squared_log_error(truth[:,2], pred[:,2]))**(1/2) 
    return (rmsle1 + rmsle2 + rmsle3)/3

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
sample_submission.head()

In [ ]:
columns = test.columns[1:]
columns

In [ ]:
X = train[columns].values
X_test = test[columns].values
target_0 = train['target_carbon_monoxide'].values.reshape(-1,1)
target_1 = train['target_benzene'].values.reshape(-1,1)
target_2 = train['target_nitrogen_oxides'].values.reshape(-1,1)

In [ ]:
target_0.min()

In [ ]:
target_1.min()

In [ ]:
target_2.min()

In [ ]:
target_0.max()

In [ ]:
target_1.max()

In [ ]:
target_2.max()

In [ ]:
train_oof = np.zeros((train.shape[0],3))
test_preds = np.zeros((test.shape[0],3))
train_oof.shape

In [ ]:
test_preds.shape

In [ ]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=137, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_features = X[train_index]
    train_target_0 = target_0[train_index]
    train_target_1 = target_1[train_index]
    train_target_2 = target_2[train_index]
    
    val_features = X[val_index]
    val_target_0 = target_0[val_index]
    val_target_1 = target_1[val_index]
    val_target_2 = target_2[val_index]
    
    model = Ridge(alpha=2500)
    model.fit(train_features, train_target_0)
    val_pred_0 = model.predict(val_features)
    train_oof[val_index,0] = val_pred_0.flatten()
    test_preds[:,0] += model.predict(X_test).flatten()/n_splits
    
    model = Ridge(alpha=2500)
    model.fit(train_features, train_target_1)
    val_pred_1 = model.predict(val_features)
    train_oof[val_index,1] = val_pred_1.flatten()
    test_preds[:,1] += model.predict(X_test).flatten()/n_splits
    
    model = Ridge(alpha=2500)
    model.fit(train_features, train_target_2)
    val_pred_2 = model.predict(val_features)
    train_oof[val_index,2] = val_pred_2.flatten()
    test_preds[:,2] += model.predict(X_test).flatten()/n_splits


In [ ]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [ ]:
train_oof

In [ ]:
train_oof[:,2].max()

In [ ]:
train_oof[:,0] = np.clip(train_oof[:,0], 0.12, None)
train_oof[:,1] = np.clip(train_oof[:,1], 0.12, None)
train_oof[:,2] = np.clip(train_oof[:,2], 2.50, None)

In [ ]:
test_preds[:,0] = np.clip(test_preds[:,0], 0.12, None)
test_preds[:,1] = np.clip(test_preds[:,1], 0.12, None)
test_preds[:,2] = np.clip(test_preds[:,2], 2.50, None)

In [ ]:
train_oof.min()

In [ ]:
target = np.hstack([target_0, target_1, target_2])
target.shape

In [ ]:
CRMSLE(target, train_oof)

In [ ]:
sample_submission[sample_submission.columns[1:]] = test_preds
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission_rr_2.csv', index=False)